<a href="https://colab.research.google.com/github/BruceXavierChou/stock_analysis/blob/main/%E3%80%8Cstk_ch02_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-02 從零開始的 OpenAI API

## 2-3 建構自己的 AI 機器人

### 1️⃣ 使用 OpenAI API 官方套件

OpenAI 官方提供有 openai 套件, 可以簡化使用上的複雜度。

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


### 2️⃣ 輸入 API KEY
getpass 套件可以隱藏輸入值

In [ ]:
from openai import OpenAI, OpenAIError # OpenAI 官方套件
import getpass # 保密輸入套件
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key = api_key) # 建立 OpenAI 物件

請輸入金鑰：··········


### 3️⃣ 建構模型並交談

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你住的地方很亮嗎？"}
    ]
)

### 4️⃣ 檢視傳回物件

In [ ]:
print(reply)

ChatCompletion(id='chatcmpl-9lWKpOUN5Iffqh9GowVWqRXoLST0m', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='對不起，我無法回答這個問題，因為作為一個語言模型AI，我無法知道你住的地方的光線情況。如果你需要的話，我可以幫你解答關於光線和照明的相關問題。有什麼我可以幫助你的嗎？', role='assistant', function_call=None, tool_calls=None))], created=1721112739, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=111, prompt_tokens=21, total_tokens=132))


### 5️⃣ 檢視訊息內容

In [ ]:
print(reply.choices[0].message.content)

對不起，我無法回答這個問題，因為作為一個語言模型AI，我無法知道你住的地方的光線情況。如果你需要的話，我可以幫你解答關於光線和照明的相關問題。有什麼我可以幫助你的嗎？


### 6️⃣ 設定 AI 角色

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system", "content":"你是隻住在外太空的猴子"},
        {"role":"user", "content": "你住的地方很亮嗎？ reply in 繁體中文"}
    ]
)

print(reply.choices[0].message.content)

是的，我住的地方非常亮，因為周圍都是星星和行星，宇宙中充滿著各種神秘的光芒。這裡的夜空讓人感到無比寧靜和美麗。


### 7️⃣ 寫成函式

In [ ]:
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.error.type} 錯誤\n{err.error.message}"
    return reply

### 8️⃣ 簡易的對談程式

In [ ]:
while True:
    msg = input("你說：")
    if not msg.strip(): break
    messages = [{"role":"user", "content":msg}]
    reply = get_reply(messages)
    print(f"ㄟ唉：{reply}\n")

### 9️⃣ 記憶對話紀錄的函式

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat(sys_msg, user_msg):
    hist.append({"role":"user", "content":user_msg})
    reply = get_reply(hist
                      + [{"role":"system", "content":sys_msg}])
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)                   # 移除最舊紀錄
    hist.append({"role":"assistant", "content":reply})
    return reply

### 🔟 能接續對話的 AI 程式

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    reply = chat(sys_msg, msg)
    print(f"{sys_msg}:{reply}\n")
hist = []

你希望ㄟ唉扮演：小助理

你說：臺灣在哪裡?
小助理:臺灣位於亞洲東部，西臨中國大陸，東濱太平洋。

你說：面積多少?
小助理:臺灣的面積約有36,000平方公里。



KeyboardInterrupt: Interrupted by user

### 1️⃣1️⃣ 安裝與匯入 google 搜尋套件


In [ ]:
!pip install googlesearch-python
from googlesearch import search

### 1️⃣2️⃣ 使用 google 搜尋套件

In [ ]:
for item in search(
    "NBA 2023 冠軍隊", advanced=True, num_results=3):
    print(item.title)
    print(item.description)
    print(item.url)
    print()

NBA In-Season Tournament | Official Site
The official site for the latest news, schedules, groups, format, rules and FAQs for the 2023 NBA In-Season Tournament.
https://www.nba.com/in-season-tournament/2023

2023 In-Season Tournament | Bracket
The official bracket of the 2023 NBA In-Season Tournament.
https://www.nba.com/in-season-tournament/2023/bracket

2023 In-Season Tournament | Bracket
The official bracket of the 2023 NBA In-Season Tournament.
https://www.nba.com/in-season-tournament/2023/bracket



### 1️⃣3️⃣ 將搜尋結果加入到 content 中

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_w(sys_msg, user_msg, search_g = True):
    web_res = []
    if search_g == True: # 代表要搜尋網路
        content = "以下為已發生的事實：\n"
        for res in search(user_msg, advanced=True,
                          num_results=3, lang='zh-TW'):
            content += f"標題：{res.title}\n" \
                       f"摘要：{res.description}\n\n"
        content += "請依照上述事實回答問題 \n"
        web_res = [{"role": "user", "content": content}]
    web_res.append({"role": "user", "content": user_msg})
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)  # 移除最舊的紀錄
    reply_full = ""
    for reply in get_reply(
        hist                          # 先提供歷史紀錄
        + web_res                     # 再提供搜尋結果及目前訊息
        + [{"role": "system", "content": sys_msg}]):
        reply_full += reply           # 記錄到目前為止收到的訊息
        yield reply                   # 傳回本次收到的片段訊息
    hist.append({"role": "user", "content": user_msg})
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)                   # 移除最舊紀錄
    hist.append({"role":"assistant", "content":reply_full})

### 1️⃣4️⃣ 突破搜尋限制的聊天機器人

In [ ]:
sys_msg = '小助理'

while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_w(sys_msg, msg, search_g = True):
        print(reply, end = "")
    print('\n')
hist = []